transformer: https://colab.research.google.com/github/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb#scrollTo=YehEMfsU3Brr

lstm: https://www.analyticsvidhya.com/blog/2021/06/natural-language-processing-sentiment-analysis-using-lstm/

In [2]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.tokenize import RegexpTokenizer
import re
import neattext.functions as nf
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import pickle

## 1. To classify the attitude that Russia will win/lose the war (Enhanced Classification)

In [39]:
with open('will_russia_win/no.txt','r') as f:
    s = f.read().split('\n')
    s = [ss for ss in s if ss!='']
    s = [nltk.tokenize.sent_tokenize(ss) for ss in s]
    no = [item for sublist in s for item in sublist]
with open('will_russia_win/yes.txt','r') as f:
    s = f.read().split('\n')
    s = [ss for ss in s if ss!='']
    s = [nltk.tokenize.sent_tokenize(ss) for ss in s]
    yes = [item for sublist in s for item in sublist]

In [40]:
stopword = stopwords.words('english')
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'\w+')
tags = r"@\w*"
def cleanText(text):
    text = re.sub(r'@russia','russia ',text)
    text = re.sub(r'@Russia','russia ',text)
    text = re.sub(r'@ukraine','ukraine ',text)
    text = re.sub(r'@Ukraine','ukraine ',text)
    text = re.sub(r'@nato','nato ',text)
    text = re.sub(r'@Nato','nato ',text)  
    text = re.sub(r'@NATO','nato ',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    text = re.sub(r'/',' ',text)
    text = re.sub(r'\n',' ',text)
    return text

def clean_word(words):
    lancaster=LancasterStemmer()
    word_list = []
    for word in words:
        if not word in stopwords.words('english'):
            if not bool(re.search(r'\d', word)):
                word_list.append(lancaster.stem(word))
    return word_list

def list_to_str(l):
    res = ''
    for s in l:
        res = res +' '+s
    return res.strip()
def processList(yes):
    yes = [cleanText(x) for x in yes]
    yes=pd.Series(yes)
    yes = yes.apply(nf.remove_urls)
    yes = yes.apply(nf.remove_userhandles)
    yes = yes.apply(nf.remove_hashtags)
    yes = yes.apply(nf.remove_puncts)
    yes = yes.apply(nf.remove_emojis)
    yes = yes.apply(nf.remove_special_characters)
    yes = yes.apply(nf.remove_multiple_spaces)
    yes = yes.str.lower()
    yes = [word_tokenize(i) for i in yes]
    yes = [clean_word(i) for i in yes]
    yes = [y for y in yes if len(y)>2]
    yes = [list_to_str(i) for i in yes]
    return yes

In [41]:
yes = processList(yes)
no = processList(no)

In [42]:
yes_label = [1]*len(yes)
no_label = [0]*len(no)

In [43]:
text = yes+no
label = yes_label+no_label

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
tf=TfidfVectorizer()
text_tf= tf.fit_transform(text)
X_train, X_test, y_train, y_test = train_test_split(text_tf, label, test_size=0.2, random_state=114514)
X_train, X_test = X_train.toarray(), X_test.toarray()

NameError: name 'text' is not defined

In [45]:
with open('tf-pkl/attitude.pkl','wb') as f:
    pickle.dump(tf,f)

In [13]:
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_params = {
    'n_neighbors':list(range(1,20,2))+list(range(20,105,5)),
    'weights':['uniform', 'distance']
}

from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic_params = {
    'penalty':['l1','l2','elasticnet','none']
}


from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_params = {
    'n_estimators':list(range(2,21,2)),
    'criterion':['gini','entropy'],
}


from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc_params = {
    'criterion':['gini', 'entropy'],
    'splitter':['best','random']
}

from sklearn.svm import SVC
rbf_svc = SVC()
rbf_svc_params = {
    'C':[0.01,0.1,1,10]
}

from sklearn.svm import SVC
lin_svc = SVC()
lin_svc_params = {
    'kernel':['linear'],
    'C':[0.01,0.1,1,10]
}


from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb_params = {
    'alpha':[0,0.01,0.1,1,10]
}

from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc_params = {
    'loss':['deviance','exponential'],
    'learning_rate':[0.0001,0.001,0.01,0.1],
    'n_estimators':[2,10,50,100,200],
    'criterion':['friedman_mse','squared_error','mse','mae']
}


from sklearn.ensemble import  AdaBoostClassifier
abc = AdaBoostClassifier()
abc_params={
    'learning_rate':[0.0001,0.001,0.01,0.1],
    'n_estimators':[2,10,50,100,200],
}


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda_params = {
    'solver':['svd','lsqr','eigen']
}

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda_params = {
    'reg_param':[0,0.001,0.001,0.01,0.1]
}

from sklearn.neural_network import MLPClassifier
nn = MLPClassifier()
nn_params = {
    'alpha':[0.01,0.1,1],
    'max_iter':[2000]
}



classifiers = {
    'knn':knn,
    'logistic_regression':logistic,
    'random_forest':rfc,
    'decision_tree':dtc,
    'rbf_svc':rbf_svc,
    'linear_svc':lin_svc,
    'naive_bayes':mnb,
    'adaboost':abc,
    'linear_discriminant':lda,
    'quadratic_discriminant':qda,
    'neural_network':nn
    
    
}
params = {
    'knn':knn_params,
    'logistic_regression':logistic_params,
    'random_forest':rfc_params,
    'decision_tree':dtc_params,
    'rbf_svc':rbf_svc_params,
    'linear_svc':lin_svc_params,
    'naive_bayes':mnb_params,
    'adaboost':abc_params,
    'linear_discriminant':lda_params,
    'quadratic_discriminant':qda_params,
    'neural_network':nn_params
}

best={}

In [4]:
from tqdm.notebook import tqdm
for k in tqdm(classifiers):
    gcv = GridSearchCV(estimator=classifiers[k], param_grid=params[k], n_jobs=-1)
    gcv.fit(X_train, y_train)
    best[k]=gcv.best_estimator_
    print(k)

  0%|          | 0/11 [00:00<?, ?it/s]

NameError: name 'X_train' is not defined

In [21]:
score_dict = {}
for k in tqdm(best):
    with open(f'sklearn-pkl/attitude/{k}.pkl','wb') as f:
        pickle.dump(best[k],f)
    y_pred = best[k].predict(X_test)
    score_dict[k]={
        'accuracy':accuracy_score(y_pred,y_test),
        'precision':precision_score(y_pred,y_test),
        'recall':recall_score(y_pred,y_test),
        'f1':f1_score(y_pred,y_test),
        'Details':str(best[k])
    }

  0%|          | 0/11 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
attitude_result = pd.DataFrame(score_dict).T
attitude_result.to_csv('attitude_result.csv')
attitude_result

,accuracy,precision,recall,f1,Details
knn,0.703704,0.423077,0.55,0.478261,KNeighborsClassifier(n_neighbors=1)
logistic_regression,0.753086,0.384615,0.714286,0.5,LogisticRegression(penalty='none')
random_forest,0.703704,0.076923,1.0,0.142857,RandomForestClassifier(n_estimators=14)
decision_tree,0.728395,0.461538,0.6,0.521739,"DecisionTreeClassifier(criterion='entropy', sp..."
rbf_svc,0.679012,0.076923,0.5,0.133333,SVC(C=10)
linear_svc,0.753086,0.269231,0.875,0.411765,"SVC(C=1, kernel='linear')"
naive_bayes,0.679012,0.384615,0.5,0.434783,MultinomialNB(alpha=0.1)
adaboost,0.679012,0.0,0.0,0.0,"AdaBoostClassifier(learning_rate=0.0001, n_est..."
linear_discriminant,0.703704,0.423077,0.55,0.478261,LinearDiscriminantAnalysis()
quadratic_discriminant,0.679012,0.0,0.0,0.0,QuadraticDiscriminantAnalysis(reg_param=0.001)


## 2. Spatial Classification (Enhanced Classification)

In [3]:
df1 = pd.read_csv('data_with_city.csv').iloc[:,1:]

In [4]:
import re
import nltk
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text
df1['cleaned'] = df1['Text'].apply(clean)
from tqdm.notebook import tqdm
tqdm.pandas()

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

df1['tagged'] = df1['cleaned'].progress_apply(token_stop_pos)

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

df1['lemma'] = df1['tagged'].progress_apply(lemmatize)

from nltk.stem import LancasterStemmer
lancaster = LancasterStemmer()
def stemmize(tagged):
    stemmed = ''
    for word, _ in tagged:
        stemmed = stemmed +' '+lancaster.stem(word)
    return stemmed.strip()

df1['stemmed'] = df1['tagged'].progress_apply(stemmize)

[nltk_data] Downloading package punkt to /Users/zhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/zhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zhang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


  0%|          | 0/12181 [00:00<?, ?it/s]

  0%|          | 0/12181 [00:00<?, ?it/s]

  0%|          | 0/12181 [00:00<?, ?it/s]

In [5]:
df1 = df1.iloc[:,4:]

In [8]:
tf=TfidfVectorizer()
text_tf= tf.fit_transform(df1.Text)
X_train, X_test, y_train, y_test = train_test_split(text_tf, df1.country, test_size=0.2, random_state=114514)
X_train, X_test = X_train.toarray(), X_test.toarray()

In [9]:
with open('tf-pkl/geo.pkl','wb') as f:
    pickle.dump(tf,f)

In [10]:
X_train = X_train[:,:5000]
X_test = X_test[:,:5000]

In [14]:
knn = KNeighborsClassifier(n_neighbors=10)
logistic = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=10)
dtc = tree.DecisionTreeClassifier()
rbf_svc = SVC(C=0.125)
lin_svc = SVC(kernel='linear',C=1)
mnb = MultinomialNB(alpha=0.1)
abc = AdaBoostClassifier()
nn = MLPClassifier(alpha=0.1,max_iter=2000)
classifiers = {
    'knn':knn,
    'logistic_regression':logistic,
    'random_forest':rfc,
    'decision_tree':dtc,
    # 'rbf_svc':rbf_svc,
    # 'linear_svc':lin_svc,
    'naive_bayes':mnb,
    'adaboost':abc,
    # 'linear_discriminant':lda,
    # 'quadratic_discriminant':qda,
    # 'neural_network':nn
}


In [19]:
for k in tqdm(classifiers):
    classifiers[k]=classifiers[k].fit(X_train,y_train)
    print(k)

  0%|          | 0/6 [00:00<?, ?it/s]

knn


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logistic_regression
random_forest
decision_tree
naive_bayes
adaboost


In [13]:
del classifiers['linear_discriminant']
del classifiers['quadratic_discriminant']
classifiers['neural_network']=classifiers['neural_network'].fit(X_train,y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [14]:
del classifiers['neural_network']

In [16]:
X_train.shape

(9744, 5000)

In [17]:
classifiers

{'knn': KNeighborsClassifier(n_neighbors=10),
 'logistic_regression': LogisticRegression(),
 'random_forest': RandomForestClassifier(n_estimators=10),
 'decision_tree': DecisionTreeClassifier(),
 'naive_bayes': MultinomialNB(alpha=0.1),
 'adaboost': AdaBoostClassifier()}

In [20]:
score_dict = {}
for k in tqdm(classifiers):
    with open(f'sklearn-pkl/geo/{k}.pkl','wb') as f:
        pickle.dump(classifiers[k],f)
    y_pred = classifiers[k].predict(X_test)
    score_dict[k]={
        'accuracy':accuracy_score(y_pred,y_test),
        'Details':str(classifiers[k])
    }

  0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
geo_result = pd.DataFrame(score_dict).T
geo_result.to_csv('geo_result.csv')
geo_result

,accuracy,Details
knn,0.300369,KNeighborsClassifier(n_neighbors=10)
logistic_regression,0.354945,LogisticRegression()
random_forest,0.307755,RandomForestClassifier(n_estimators=10)
decision_tree,0.270825,DecisionTreeClassifier()
naive_bayes,0.34961,MultinomialNB(alpha=0.1)
adaboost,0.357407,AdaBoostClassifier()


In [27]:
t = tf.transform(['Fuck Russians! Your Motherfucker! Fuck Putin!'])[:,:5000]

In [28]:
classifiers['adaboost'].predict(t)

array(['GB'], dtype=object)